In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from data_cleaning import new_clean_data
from datetime import datetime
import plotly.graph_objs as go
import plotly.offline as pyo
import seaborn as sns
from data_cleaning import new_clean_data
%matplotlib inline

In [ ]:
def calculate_idle_time2(dataframe):
    data = dataframe.copy()
    earliest_datetime_2017 = datetime(2017, 1, 1, 0, 0, 0)
    # liefert bessere Ergebisse, da viele Ausreiser bei der Berechnung der idle_time ansonsten enstehen von Stationen, die erst später eingeführt wurden
    data['idle_time'] = 0

    latest_end_time = {}
    for i in range(len(data)):
        print(str(i))
        start_station = data.loc[i, 'start_station_id']
        end_station = data.loc[i, 'end_station_id']
        if start_station in latest_end_time:
            last_end_times = latest_end_time[start_station]

            for time in last_end_times:
                if time <= data.loc[i, 'start_time']:
                    idle_time = data.loc[i, 'start_time'] - time
                    data.at[i, 'idle_time'] = idle_time
                    break

            if end_station in latest_end_time:
                latest_end_time[end_station].insert(0, data.loc[i, 'end_time'])
            else:
                latest_end_time[end_station] = [data.loc[i, 'end_time']]    
        else:
            latest_end_time[end_station] = [data.loc[i, 'end_time']]

    return data

In [ ]:
bluebikes = new_clean_data(r"C:\Users\STH_0\Desktop\Data\bluebikes_2017_2018")

In [ ]:
test = calculate_idle_time2(bluebikes)
test.to_csv(r"C:\Users\STH_0\Desktop\Data\bluebikes_idle_time_v4.csv", index=False)

In [ ]:
bluebikes_idle_time = pd.read_csv(r"C:\Users\STH_0\Desktop\Data\bluebikes_idle_time_v4.csv")
bluebikes_idle_time['idle_time'] = pd.to_timedelta(bluebikes_idle_time['idle_time'])
bluebikes_idle_time['idle_time'] = bluebikes_idle_time['idle_time'].dt.total_seconds()
bluebikes_idle_time['idle_time'] = bluebikes_idle_time['idle_time'] / 60
bluebikes_idle_time.head()

In [ ]:
bluebikes_idle_time.describe()

In [ ]:
#plot idle time distribution with plotly
data_new = [go.Histogram(x=bluebikes_idle_time['idle_time'])]
layout = go.Layout(title='Idle Time Distribution', xaxis=dict(title='Idle Time in Minutes'), yaxis=dict(title='Count'))
fig = go.Figure(data=data_new, layout=layout)
pyo.iplot(fig)

In [ ]:
# Convert 'start_time' column to datetime
bluebikes_idle_time['start_time'] = pd.to_datetime(bluebikes_idle_time['start_time'])

# Group the data by date and calculate the mean of idle time for each day
daily_idle_time_mean = bluebikes_idle_time.groupby(bluebikes_idle_time['start_time'].dt.date)['idle_time'].mean()

# Group the data by date and count the number of data points for each day
daily_data_count = bluebikes_idle_time.groupby(bluebikes_idle_time['start_time'].dt.date).size()

# Create the scatter plots
trace_idle_time = go.Scatter(x=daily_idle_time_mean.index, y=daily_idle_time_mean.values, name='Mean Idle Time')
trace_data_count = go.Scatter(x=daily_data_count.index, y=daily_data_count.values, name='Data Point Count')

# Set the layout
layout = go.Layout(title='Mean Idle Time and Data Point Count over 2 Years',
                   xaxis=dict(title='Date'), yaxis=dict(title='Value'))

# Create the figure
fig = go.Figure(data=[trace_idle_time, trace_data_count], layout=layout)

# Plot the figure
pyo.iplot(fig)

In [ ]:
bluebikes_idle_time['start_time'] = pd.to_datetime(bluebikes_idle_time['start_time'])

# Extract the hour from the 'start_time' column
bluebikes_idle_time['hour_of_day'] = bluebikes_idle_time['start_time'].dt.hour

# Group the data by hour of the day and calculate the mean of idle time for each hour
hourly_idle_time_mean = bluebikes_idle_time.groupby('hour_of_day')['idle_time'].mean()

# Create the bar plot
data = [go.Bar(x=hourly_idle_time_mean.index, y=hourly_idle_time_mean.values)]

# Set the layout
layout = go.Layout(title='Mean Idle Time by Hour of the Day',
                   xaxis=dict(title='Hour of the Day', tickmode='array', tickvals=hourly_idle_time_mean.index, ticktext=hourly_idle_time_mean.index),
                   yaxis=dict(title='Mean Idle Time'))

# Create the figure
fig = go.Figure(data=data, layout=layout)

# Plot the figure
pyo.iplot(fig)

In [ ]:

bluebikes_idle_time['start_time'] = pd.to_datetime(bluebikes_idle_time['start_time'])
bluebikes_idle_time['hour_of_day'] = bluebikes_idle_time['start_time'].dt.hour

# Group the data by hour of the day and calculate the mean of idle time for each hour
daily_idle_time_mean = bluebikes_idle_time.groupby('weekday')['idle_time'].mean()
daily_data_count = bluebikes_idle_time.groupby('weekday').size()

# Create the bar plot
data = [go.Scatter(x=daily_idle_time_mean.index, y=daily_idle_time_mean.values)]
data2 = [go.Scatter(x=daily_data_count.index, y=daily_data_count.values)]
# Set the layout
layout = go.Layout(title='Mean Idle Time by Hour of the Day',
                   xaxis=dict(title='Hour of the Day'),
                   yaxis=dict(title='Mean Idle Time'))


# Create the figure
# add data2 to graph
fig = go.Figure(data=[data, data2], layout=layout)

# Plot the figure
pyo.iplot(fig)

In [ ]:
daily_idle_time = bluebikes_idle_time.groupby(bluebikes_idle_time['start_time'].dt.date)['idle_time'].mean()

# Group the data by date and count the number of data points for each day
daily_data_count = bluebikes_idle_time.groupby(bluebikes_idle_time['start_time'].dt.date).size()

# Create the scatter plot
data = [go.Scatter(x=daily_data_count, y=daily_idle_time, mode='markers')]

# Set the layout
layout = go.Layout(title='Idle Time vs Data Point Count', xaxis=dict(title='Data Point Count'), yaxis=dict(title='Idle Time'))

# Create the figure
fig = go.Figure(data=data, layout=layout)

# Plot the figure
pyo.iplot(fig)